In [33]:

import glob
import string
import os
import random
import math
from xml.dom.minidom import Document
import cv2

# Preprocess the annotations
### Sort the categories (Only one box in each png)

In [25]:
txt_list = glob.glob('./training/label_2/*.txt')


## Sort the category in the txt_list
def sort_category(txt_list):
  category_list= []
  for item in txt_list:
    with open(item) as tdf:
      for each_line in tdf:
        labeldata = each_line.strip().split(' ') # clean and split
        category_list.append(labeldata[0]) # [0] class
    print(set(category_list))

## Merge the list      
def merge(line): # line: list
  each_line = ''
  for i in range(len(line)):
    if i!= (len(line)-1):
      each_line = each_line + line[i] + ' '
    else:
      each_line = each_line + line[i]    
  each_line = each_line + '\n'
  return (each_line)



## Modify the txt_list (Reduce the number of classes) ##
for item in txt_list:
  new_txt=[]
  
  with open(item, 'r') as r_tdf:
    for each_line in r_tdf:
        labeldata = each_line.strip().split(' ')
        if labeldata[0] in ['Pedestrian','Person_sitting','Cyclist']:
            labeldata[0] = labeldata[0].replace(labeldata[0],'person')
        if labeldata[0] in ['Truck','Van','Tram']:
            labeldata[0] = labeldata[0].replace(labeldata[0],'truck')
        if labeldata[0] in ['Car']: 
            labeldata[0] = labeldata[0].replace(labeldata[0],'car')
        if labeldata[0] == 'DontCare':
            continue
        if labeldata[0] == 'Misc':
            continue
        new_txt.append(merge(labeldata))
        break
         
  with open(item,'w+') as w_tdf: # Cover
    for temp in new_txt:
        w_tdf.write(temp)

sort_category(txt_list)

{'car'}
{'car'}
{'car'}
{'car'}
{'car'}
{'car'}
{'car'}
{'car', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', 'person', 'truck'}
{'car', '

### Change the form of annotations

In [39]:

def generate_xml(name, split_lines, img_size, class_ind):
    # name: file_name(like 000001)
    # split_lines: lines in xml files
    doc = Document()  # Create DOM Object

    annotation = doc.createElement('annotation')
    doc.appendChild(annotation)

    title = doc.createElement('folder')
    title_text = doc.createTextNode('KITTI')
    title.appendChild(title_text)
    annotation.appendChild(title)

    img_name = name + '.jpg'

    title = doc.createElement('filename')
    title_text = doc.createTextNode(img_name)
    title.appendChild(title_text)
    annotation.appendChild(title)

    source = doc.createElement('source')
    annotation.appendChild(source)

    title = doc.createElement('database')
    title_text = doc.createTextNode('The KITTI Database')
    title.appendChild(title_text)
    source.appendChild(title)

    title = doc.createElement('annotation')
    title_text = doc.createTextNode('KITTI')
    title.appendChild(title_text)
    source.appendChild(title)

    size = doc.createElement('size')
    annotation.appendChild(size)

    title = doc.createElement('width')
    title_text = doc.createTextNode(str(img_size[1]))
    title.appendChild(title_text)
    size.appendChild(title)

    title = doc.createElement('height')
    title_text = doc.createTextNode(str(img_size[0]))
    title.appendChild(title_text)
    size.appendChild(title)

    title = doc.createElement('depth')
    title_text = doc.createTextNode(str(img_size[2]))
    title.appendChild(title_text)
    size.appendChild(title)

    '''
    #Values    Name      Description
    ----------------------------------------------------------------------------
    1    type         Describes the type of object: 'Car', 'Van', 'Truck',
                        'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram',
                        'Misc' or 'DontCare'
    1    truncated    Float from 0 (non-truncated) to 1 (truncated), where
                        truncated refers to the object leaving image boundaries
    1    occluded     Integer (0,1,2,3) indicating occlusion state:
                        0 = fully visible, 1 = partly occluded
                        2 = largely occluded, 3 = unknown
    1    alpha        Observation angle of object, ranging [-pi..pi]
    4    bbox         2D bounding box of object in the image (0-based index):
                        contains left, top, right, bottom pixel coordinates
    3    dimensions   3D object dimensions: height, width, length (in meters)
    3    location     3D object location x,y,z in camera coordinates (in meters)
    1    rotation_y   Rotation ry around Y-axis in camera coordinates [-pi..pi]
    1    score        Only for results: Float, indicating confidence in
                        detection, needed for p/r curves, higher is better.
    '''
    for split_line in split_lines:
        line = split_line.strip().split()
        if line[0] in class_ind:
            object_ = doc.createElement('object')
            annotation.appendChild(object_)

            title = doc.createElement('name')
            title_text = doc.createTextNode(line[0])
            title.appendChild(title_text)
            object_.appendChild(title)

            bndbox = doc.createElement('bndbox')
            object_.appendChild(bndbox)
            title = doc.createElement('xmin')
            title_text = doc.createTextNode(str(int(float(line[4]))))
            title.appendChild(title_text)
            bndbox.appendChild(title)
            title = doc.createElement('ymin')
            title_text = doc.createTextNode(str(int(float(line[5]))))
            title.appendChild(title_text)
            bndbox.appendChild(title)
            title = doc.createElement('xmax')
            title_text = doc.createTextNode(str(int(float(line[6]))))
            title.appendChild(title_text)
            bndbox.appendChild(title)
            title = doc.createElement('ymax')
            title_text = doc.createTextNode(str(int(float(line[7]))))
            title.appendChild(title_text)
            bndbox.appendChild(title)

    f = open('./xmls/' + name + '.xml', 'w') # create a new xml file
    f.write(doc.toprettyxml(indent = ''))
    f.close()


from PIL import Image
## png2jpg: https://blog.csdn.net/weixin_40446557/article/details/104059660 n*100k->N*10k
def PNG_JPG(PngPath):
    img = cv2.imread(PngPath, 0)
    w, h = img.shape[::-1]
    infile = PngPath
    outfile = os.path.splitext(infile)[0] + ".jpg"
    img = Image.open(infile)
    img = img.resize((int(w / 2), int(h / 2)), Image.ANTIALIAS)
    if len(img.split()) == 4:
        r, g, b, a = img.split()
        img = Image.merge("RGB", (r, g, b))
        img.convert('RGB').save(outfile, quality=70)
        os.remove(PngPath)
    else:
        img.convert('RGB').save(outfile, quality=70)
        os.remove(PngPath)
    return outfile
    
    
    
## Generate xml files    
class_ind = ('person', 'car', 'truck')
labels_dir = './training/label_2'
for parent, dirnames, filenames in os.walk(labels_dir): # Root dir, Sub dir and files in the root dir
    for file_name in filenames:
        if file_name == '.DS_Store': # otherwise decoding error
            continue
        full_path = os.path.join(parent, file_name)
        f = open(full_path)
        split_lines = f.readlines() # each line in each xml file
        name = file_name[:-4] # before '.txt'
            
        img_name = name + '.jpg' # 'png', not with ONG_JPG
        img_path = os.path.join('../images', img_name)
        img_size = cv2.imread(img_path).shape
        #PNG_JPG(img_path)
            
        generate_xml(name, split_lines, img_size, class_ind)

# label map and trainval.txt file

In [37]:
def get_sample_value(txt_name):
    label_path = './training/label_2/'
    txt_path = label_path + txt_name + '.txt'
    with open(txt_path) as r_tdf:
        labeldata = r_tdf.read().strip().split(' ')
        if labeldata[0] == 'car':
            return ' 3'
        elif labeldata[0] == 'truck':
            return ' 2'
        elif labeldata[0] == 'person':
            return ' 1'

txt_list_path =  glob.glob('./training/label_2/*.txt')
txt_list = []
for item in txt_list_path:
    temp1, temp2 = os.path.splitext(os.path.basename(item))
    txt_list.append(temp1)
txt_list.sort() # ['000000', '000001', '000002', ...]


category_name = ['person','car','truck']

trainval_txt_name = './trainval.txt'
with open(trainval_txt_name, 'w') as w_tdf:
    for item in txt_list:
        w_tdf.write(item + ' ' + get_sample_value(item) + '\n')
